In [2]:
from keras.datasets import fashion_mnist
from matplotlib import pyplot as plt


(x_train,y_train),(x_test,y_test)=fashion_mnist.load_data()
x_train = x_train/255
x_test = x_test/255

In [20]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

class Neural_network:
    np.random.seed(10)
    def __init__(self,x_train,y_train,input_dim,hidden_layers_size,hidden_layers,output_dim,batch_size=30,epochs=10,activation_func="relu"
           ,learning_rate=6e-3 ,decay_rate=0.9,beta=0.9,beta1=0.9,beta2=0.99,optimizer="adam",weight_init="xavier",loss="cross_entropy"):

        self.x_train,self.x_cv,self.y_train,self.y_cv = train_test_split(x_train, y_train, test_size=0.10, random_state=100,stratify=y_train)

        np.random.seed(10)
        self.input_dim = input_dim
        self.hidden_layers = hidden_layers
        self.hidden_layers_size = hidden_layers_size
        self.output_dim = output_dim

        self.batch = batch_size
        self.epochs = epochs
        self.activation_func = activation_func
        self.learning_rate = learning_rate
        self.decay_rate = decay_rate
        self.optimizer = optimizer
        self.weight_init = weight_init
        self.beta = beta
        self.beta1 = beta1
        self.beta2 = beta2
        self.loss = loss

        self.layers = [self.input_dim] + self.hidden_layers*[self.hidden_layers_size] + [self.output_dim]

        layers = self.layers.copy()
        self.weights = []
        self.biases = []
        self.activations = []
        self.activation_gradients = []
        self.weights_gradients = []
        self.biases_gradients = []

        for i in range(len(layers)-1):
            if self.weight_init == 'xavier':
                std = np.sqrt(2/(layers[i]*layers[i+1]))
                self.weights.append(np.random.normal(0,std,(layers[i],layers[i+1])))
                self.biases.append(np.random.normal(0,std,(layers[i+1])))
            else:
                self.weights.append(np.random.normal(0,0.5,(layers[i],layers[i+1])))
                self.biases.append(np.random.normal(0,0.5,(layers[i+1])))
            self.activations.append(np.zeros(layers[i]))
            self.activation_gradients.append(np.zeros(layers[i+1]))
            self.weights_gradients.append(np.zeros((layers[i],layers[i+1])))
            self.biases_gradients.append(np.zeros(layers[i+1]))

        self.activations.append(np.zeros(layers[-1]))
        
        if optimizer == 'adam':
            self.adam(self.x_train,self.y_train)
        elif optimizer == 'sgd':
            self.sgd(self.x_train,self.y_train)
        elif optimizer == 'momentum':
            self.momentum(self.x_train,self.y_train)
        elif optimizer == 'nesterov':
            self.nesterov(self.x_train,self.y_train)
        elif optimizer == 'nadam':
            self.nadam(self.x_train,self.y_train)
        elif optimizer == 'rmsprop':
            self.rmsprop(self.x_train,self.y_train)


    def sigmoid(self,activations):
        res = []
        for z in activations:
            if z>40:
                res.append(1.0)
            elif z<-40:
                res.append(0.0)
            else:
                res.append(1/(1+np.exp(-z)))
        return np.asarray(res)

    def tanh(self,activations):
        res = []
        for z in activations:
            if z>20:
                res.append(1.0)
            elif z<-20:
                res.append(-1.0)
            else:
                res.append((np.exp(z) - np.exp(-z))/(np.exp(z) + np.exp(-z)))
        return np.asarray(res)

    def relu(self,activations):
        res = []
        for i in activations:
            if i<= 0:
                res.append(0)
            else:
                res.append(i)
        return np.asarray(res)

    def softmax(self,activations):
        tot = 0
        for z in activations:
            tot += np.exp(z)
        return np.asarray([np.exp(z)/tot for z in activations])

    def onehot(self,y):
      y_onehot = np.zeros(self.output_dim)
      y_onehot[y] = 1
      return y_onehot


    def forward_propagation(self,x,y,weights,biases):
        self.activations[0] = x
        n = len(self.layers)
        for i in range(n-2):
            if self.activation_func == "sigmoid":
                self.activations[i+1] = self.sigmoid(np.matmul(weights[i].T,self.activations[i])+biases[i])
            elif self.activation_func == "tanh":
                self.activations[i+1] = self.tanh(np.matmul(weights[i].T,self.activations[i])+biases[i])
            elif self.activation_func == "relu":
                self.activations[i+1] = self.relu(np.matmul(weights[i].T,self.activations[i])+biases[i])

        self.activations[n-1] = self.softmax(np.matmul(weights[n-2].T,self.activations[n-2])+biases[n-2])  
        if self.loss == "cross_entropy":      
          return -(np.log2(self.activations[-1][y])) #Return cross entropy loss for single data point.
        elif self.loss == "squared_loss":
          y_onehot = self.onehot(y)
          return np.sum(np.square(self.activations[-1] - y_onehot))

    def grad_w(self,i):
        return np.matmul(self.activations[i].reshape((-1,1)),self.activation_gradients[i].reshape((1,-1)))


    def grad_b(self,i):
        return self.activation_gradients[i]


    def backward_propagation(self,x,y,weights,biases):
        y_onehot = self.onehot(y)
        n = len(self.layers)

        if self.loss == "cross_entropy": 
          self.activation_gradients[-1] =  -1*(y_onehot - self.activations[-1])
        elif self.loss == "squared_loss":
          temp_vec = 2 * (self.activations[-1] - y_onehot) * (self.activations[-1])
          for i in range(len(self.activations[-1])):
            self.activation_gradients[-1][i] = np.dot(temp_vec,(self.onehot(i) - np.asarray([self.activations[-1][i]]*self.output_dim)))
          
        for i in range(n-2,-1,-1):
            self.weights_gradients[i] += self.grad_w(i)
            self.biases_gradients[i] += self.grad_b(i)
            if i!=0:
                value = np.matmul(weights[i],self.activation_gradients[i])
                if self.activation_func == "sigmoid":
                    self.activation_gradients[i-1] = value * self.activations[i] * (1-self.activations[i])
                elif self.activation_func == "tanh":
                    self.activation_gradients[i-1] = value * (1-np.square(self.activations[i]))
                elif self.activation_func == "relu":
                    res = []
                    for k in self.activations[i]:
                        if k>0: res.append(1.0)
                        else: res.append(0.0)
                    res = np.asarray(res)
                    self.activation_gradients[i-1] = value * res

    def gradient_descent(self,x_train,y_train):
        for i in range(self.epochs):
            print('Epoch---',i+1,end=" ")
            loss = 0

            self.weights_gradients = [0*i for i in (self.weights_gradients)]
            self.biases_gradients = [0*i for i in (self.biases_gradients)]
            
            index = 1
            for x,y in zip(x_train,y_train):
                x = x.ravel()
                loss += self.forward_propagation(x,y,self.weights,self.biases)
                self.backward_propagation(x,y,self.weights,self.biases)

                if index % self.batch == 0 or index == x_train.shape[0]:
                    for j in range(len(self.weights)):
                        self.weights[j] -= self.learning_rate * self.weights_gradients[j]
                        self.biases[j] -= self.learning_rate * self.biases_gradients[j]
                    self.weights_gradients = [0*i for i in (self.weights_gradients)]
                    self.biases_gradients = [0*i for i in (self.biases_gradients)]
                index += 1 

            for x,y in zip(self.x_cv,self.y_cv):
               x=x.ravel()
               val_loss+=self.forward_propagation(x,y,self.weights,self.biases)

            acc=round(self.calculate_accuracy(x_train,y_train),3)
            val_acc=round(self.calculate_accuracy(self.x_cv,self.y_cv),3)
            # wandb.log({'train_loss':loss/x_train.shape[0],'train_accuracy':acc,'val_loss':val_loss/self.x_cv.shape[0],'val_accuarcy':val_acc})
            print('  loss = ',loss/x_train.shape[0],'  accuracy = ',acc,'   validation loss= ',val_loss/self.x_cv.shape[0],'  validation accuaracy= ',val_acc)


    def sgd(self,x_train,y_train):
        for i in range(self.epochs):
            print('Epoch---',i+1,end=" ")
            loss = 0
            val_loss=0

            index = 1
            for x,y in zip(x_train,y_train):
                x = x.ravel()
                loss += self.forward_propagation(x,y,self.weights,self.biases)
                self.backward_propagation(x,y,self.weights,self.biases)

                if index % self.batch == 0 or index == x_train.shape[0]:
                    for j in range(len(self.weights)):
                        self.weights[j] -= self.learning_rate * self.weights_gradients[j]
                        self.biases[j] -= self.learning_rate * self.biases_gradients[j]
                    self.weights_gradients = [0*i for i in (self.weights_gradients)]
                    self.biases_gradients = [0*i for i in (self.biases_gradients)]
                index +=1

            for x,y in zip(self.x_cv,self.y_cv):
               x=x.ravel()
               val_loss+=self.forward_propagation(x,y,self.weights,self.biases)

            acc=round(self.calculate_accuracy(x_train,y_train),3)
            val_acc=round(self.calculate_accuracy(self.x_cv,self.y_cv),3)
            # wandb.log({'train_loss':loss/x_train.shape[0],'train_accuracy':acc,'val_loss':val_loss/self.x_cv.shape[0],'val_accuarcy':val_acc})
            print('  loss = ',loss/x_train.shape[0],'  accuracy = ',acc,'   validation loss= ',val_loss/self.x_cv.shape[0],'  validation accuaracy= ',val_acc)

    
    def momentum(self,x_train,y_train):
        prev_gradients_w = [0*i for i in (self.weights_gradients)]
        prev_gradients_b = [0*i for i in (self.biases_gradients)]

        for i in range(self.epochs):
            print('Epoch---',i+1,end=" ")
            loss = 0
            val_loss=0

            self.weights_gradients = [0*i for i in (self.weights_gradients)]
            self.biases_gradients = [0*i for i in (self.biases_gradients)]

            index = 1
            for x,y in zip(x_train,y_train):
                x = x.ravel()
                loss += self.forward_propagation(x,y,self.weights,self.biases)
                self.backward_propagation(x,y,self.weights,self.biases)
                if index % self.batch == 0 or index == x_train.shape[0]:
                    for j in range(len(self.weights)):
                        v_w = (self.decay_rate * prev_gradients_w[j] + self.learning_rate * self.weights_gradients[j])
                        v_b = (self.decay_rate * prev_gradients_b[j] + self.learning_rate * self.biases_gradients[j])
                        self.weights[j] -= v_w
                        self.biases[j] -= v_b
                        prev_gradients_w[j] = v_w
                        prev_gradients_b[j] = v_b
                    self.weights_gradients = [0*i for i in (self.weights_gradients)]
                    self.biases_gradients = [0*i for i in (self.biases_gradients)]

                index +=1
            for x,y in zip(self.x_cv,self.y_cv):
               x=x.ravel()
               val_loss+=self.forward_propagation(x,y,self.weights,self.biases)

            acc=round(self.calculate_accuracy(x_train,y_train),3)
            val_acc=round(self.calculate_accuracy(self.x_cv,self.y_cv),3)
            # wandb.log({'train_loss':loss/x_train.shape[0],'train_accuracy':acc,'val_loss':val_loss/self.x_cv.shape[0],'val_accuarcy':val_acc})
            print('  loss = ',loss/x_train.shape[0],'  accuracy = ',acc,'   validation loss= ',val_loss/self.x_cv.shape[0],'  validation accuaracy= ',val_acc)


    def nesterov(self,x_train,y_train):
        prev_gradients_w = [0*i for i in (self.weights_gradients)]
        prev_gradients_b = [0*i for i in (self.biases_gradients)]

        for i in range(self.epochs):
            print('Epoch---',i+1,end=" ")
            loss = 0
            val_loss=0

            weights = [self.weights[j] -  self.decay_rate * prev_gradients_w[j] for j in range(len(self.weights))]
            biases = [self.biases[j] -  self.decay_rate * prev_gradients_b[j] for j in range(len(self.biases))]

            self.weights_gradients = [0*j for j in (self.weights_gradients)]
            self.biases_gradients = [0*j for j in (self.biases_gradients)]
            index = 1
            for x,y in zip(x_train,y_train):
                x = x.ravel()
                loss += self.forward_propagation(x,y,weights,biases)
                self.backward_propagation(x,y,weights,biases)
                if index % self.batch == 0 or index == x_train.shape[0]:
                    for j in range(len(self.weights)):
                        prev_gradients_w[j] = self.decay_rate * prev_gradients_w[j] + self.learning_rate*self.weights_gradients[j] 
                                           
                        prev_gradients_b[j] = self.decay_rate * prev_gradients_b[j] + self.learning_rate*self.biases_gradients[j] 
                                        
                        self.weights[j] -= prev_gradients_w[j]
                        self.biases[j] -= prev_gradients_b[j]

                    weights = [self.weights[j] -  self.decay_rate * prev_gradients_w[j] for j in range(len(self.weights))]
                    biases = [self.biases[j] -  self.decay_rate * prev_gradients_b[j] for j in range(len(self.biases))]

                    self.weights_gradients = [0*j for j in (self.weights_gradients)]
                    self.biases_gradients = [0*j for j in (self.biases_gradients)]
                    
                index += 1
            for x,y in zip(self.x_cv,self.y_cv):
               x=x.ravel()
               val_loss+=self.forward_propagation(x,y,self.weights,self.biases)

            acc=round(self.calculate_accuracy(x_train,y_train),3)
            val_acc=round(self.calculate_accuracy(self.x_cv,self.y_cv),3)
            # wandb.log({'train_loss':loss/x_train.shape[0],'train_accuracy':acc,'val_loss':val_loss/self.x_cv.shape[0],'val_accuarcy':val_acc})
            print('  loss = ',loss/x_train.shape[0],'  accuracy = ',acc,'   validation loss= ',val_loss/self.x_cv.shape[0],'  validation accuaracy= ',val_acc)


    def rmsprop(self,x_train,y_train):
        prev_gradients_w = [0*i for i in (self.weights_gradients)]
        prev_gradients_b = [0*i for i in (self.biases_gradients)]
        eps = 1e-2

        for i in range(self.epochs):
            print('Epoch---',i+1,end=" ")
            loss = 0
            val_loss=0

            self.weights_gradients = [0*i for i in (self.weights_gradients)]
            self.biases_gradients = [0*i for i in (self.biases_gradients)]

            index = 1
            for x,y in zip(x_train,y_train):
                x = x.ravel()
                loss += self.forward_propagation(x,y,self.weights,self.biases)
                self.backward_propagation(x,y,self.weights,self.biases)
                if index % self.batch == 0 or index == x_train.shape[0]:
                    for j in range(len(self.weights)):
                        v_w = (self.beta * prev_gradients_w[j] + (1-self.beta) * np.square(self.weights_gradients[j]))
                        v_b = (self.beta * prev_gradients_b[j] + (1-self.beta) * np.square(self.biases_gradients[j]))
                        self.weights[j] -= self.learning_rate * (self.weights_gradients[j] /(np.sqrt(v_w + eps)))
                        self.biases[j] -= self.learning_rate * (self.biases_gradients[j] /(np.sqrt(v_b + eps)))
                        prev_gradients_w[j] = v_w
                        prev_gradients_b[j] = v_b
                    self.weights_gradients = [0*i for i in (self.weights_gradients)]
                    self.biases_gradients = [0*i for i in (self.biases_gradients)]

                index +=1
            for x,y in zip(self.x_cv,self.y_cv):
               x=x.ravel()
               val_loss+=self.forward_propagation(x,y,self.weights,self.biases)

            acc=round(self.calculate_accuracy(x_train,y_train),3)
            val_acc=round(self.calculate_accuracy(self.x_cv,self.y_cv),3)
            # wandb.log({'train_loss':loss/x_train.shape[0],'train_accuracy':acc,'val_loss':val_loss/self.x_cv.shape[0],'val_accuarcy':val_acc})
            print('  loss = ',loss/x_train.shape[0],'  accuracy = ',acc,'   validation loss= ',val_loss/self.x_cv.shape[0],'  validation accuaracy= ',val_acc)


    def adam(self,x_train,y_train):
        m_prev_gradients_w = [0*i for i in (self.weights_gradients)]
        m_prev_gradients_b = [0*i for i in (self.biases_gradients)]
        v_prev_gradients_w = [0*i for i in (self.weights_gradients)]
        v_prev_gradients_b = [0*i for i in (self.biases_gradients)]

        iter = 1

        for i in range(self.epochs):
            print('Epoch---',i+1,end=" ")
            loss = 0
            val_loss=0
            eps = 1e-2
            self.weights_gradients = [0*i for i in (self.weights_gradients)]
            self.biases_gradients = [0*i for i in (self.biases_gradients)]

            index = 1
            for x,y in zip(x_train,y_train):
                x = x.ravel()
                loss += self.forward_propagation(x,y,self.weights,self.biases)
                self.backward_propagation(x,y,self.weights,self.biases)
                if index % self.batch == 0 or index == x_train.shape[0]:
                    for j in range(len(self.weights)):
                        m_w = (self.beta1 * m_prev_gradients_w[j] + (1-self.beta1) * self.weights_gradients[j])
                        m_b = (self.beta1 * m_prev_gradients_b[j] + (1-self.beta1) * self.biases_gradients[j])
                        v_w = (self.beta2 * v_prev_gradients_w[j] + (1-self.beta2) * np.square(self.weights_gradients[j]))
                        v_b = (self.beta2 * v_prev_gradients_b[j] + (1-self.beta2) * np.square(self.biases_gradients[j]))

                        m_hat_w = (m_w)/(1-(self.beta1)**iter) 
                        m_hat_b = (m_b)/(1-(self.beta1)**iter) 

                        v_hat_w = (v_w)/(1-(self.beta2)**iter) 
                        v_hat_b = (v_b)/(1-(self.beta2)**iter)

                        self.weights[j] -= self.learning_rate * (m_hat_w/(np.sqrt(v_hat_w + eps)))
                        self.biases[j] -= self.learning_rate * (m_hat_b/(np.sqrt(v_hat_b + eps)))

                        m_prev_gradients_w[j] = m_w
                        m_prev_gradients_b[j] = m_b
                        v_prev_gradients_w[j] = v_w
                        v_prev_gradients_b[j] = v_b

                    self.weights_gradients = [0*i for i in (self.weights_gradients)]
                    self.biases_gradients = [0*i for i in (self.biases_gradients)]
                    iter += 1

                index +=1
            for x,y in zip(self.x_cv,self.y_cv):
               x=x.ravel()
               val_loss+=self.forward_propagation(x,y,self.weights,self.biases)

            acc=round(self.calculate_accuracy(x_train,y_train),3)
            val_acc=round(self.calculate_accuracy(self.x_cv,self.y_cv),3)
            # wandb.log({'train_loss':loss/x_train.shape[0],'train_accuracy':acc,'val_loss':val_loss/self.x_cv.shape[0],'val_accuarcy':val_acc})
            print('  loss = ',loss/x_train.shape[0],'  accuracy = ',acc,'   validation loss= ',val_loss/self.x_cv.shape[0],'  validation accuaracy= ',val_acc)

    def nadam(self,x_train,y_train):
        m_prev_gradients_w = [0*i for i in (self.weights_gradients)]
        m_prev_gradients_b = [0*i for i in (self.biases_gradients)]
        v_prev_gradients_w = [0*i for i in (self.weights_gradients)]
        v_prev_gradients_b = [0*i for i in (self.biases_gradients)]

        iter = 1

        for i in range(self.epochs):
            print('Epoch---',i+1,end=" ")
            loss = 0
            val_loss=0
            eps = 1e-2
            self.weights_gradients = [0*i for i in (self.weights_gradients)]
            self.biases_gradients = [0*i for i in (self.biases_gradients)]

            index = 1
            for x,y in zip(x_train,y_train):
                x = x.ravel()
                loss += self.forward_propagation(x,y,self.weights,self.biases)
                self.backward_propagation(x,y,self.weights,self.biases)
                if index % self.batch == 0 or index == x_train.shape[0]:
                    for j in range(len(self.weights)):

                        m_w = (self.beta1 * m_prev_gradients_w[j] + (1-self.beta1) * self.weights_gradients[j])
                        m_b = (self.beta1 * m_prev_gradients_b[j] + (1-self.beta1) * self.biases_gradients[j])
                        v_w = (self.beta2 * v_prev_gradients_w[j] + (1-self.beta2) * np.square(self.weights_gradients[j]))
                        v_b = (self.beta2 * v_prev_gradients_b[j] + (1-self.beta2) * np.square(self.biases_gradients[j]))

                        m_hat_w = (m_w)/(1-(self.beta1)**iter) 
                        m_hat_b = (m_b)/(1-(self.beta1)**iter) 

                        v_hat_w = (v_w)/(1-(self.beta2)**iter) 
                        v_hat_b = (v_b)/(1-(self.beta2)**iter)

                        m_dash_w = self.beta1 * m_hat_w + (1-self.beta1) * self.weights_gradients[j]
                        m_dash_b = self.beta1 * m_hat_b + (1-self.beta1) * self.biases_gradients[j]

                        self.weights[j] -= self.learning_rate * (m_dash_w/(np.sqrt(v_hat_w + eps)))
                        self.biases[j] -= self.learning_rate * (m_dash_b/(np.sqrt(v_hat_b + eps)))

                        m_prev_gradients_w[j] = m_w
                        m_prev_gradients_b[j] = m_b
                        v_prev_gradients_w[j] = v_w
                        v_prev_gradients_b[j] = v_b

                    self.weights_gradients = [0*i for i in (self.weights_gradients)]
                    self.biases_gradients = [0*i for i in (self.biases_gradients)]
                    iter += 1

                index +=1

            for x,y in zip(self.x_cv,self.y_cv):
               x=x.ravel()
               val_loss+=self.forward_propagation(x,y,self.weights,self.biases)

            acc=round(self.calculate_accuracy(x_train,y_train),3)
            val_acc=round(self.calculate_accuracy(self.x_cv,self.y_cv),3)
            # wandb.log({'train_loss':loss/x_train.shape[0],'train_accuracy':acc,'val_loss':val_loss/self.x_cv.shape[0],'val_accuarcy':val_acc})
            print('  loss = ',loss/x_train.shape[0],'  accuracy = ',acc,'   validation loss= ',val_loss/self.x_cv.shape[0],'  validation accuaracy= ',val_acc)


    def calculate_accuracy(self,X,Y):
        count = 0
        for i in range(len(X)):
            if self.predict(X[i]) == Y[i]:
                count+=1
        return count/len(X)


    def predict(self,x):
        x = x.ravel()
        self.activations[0] = x
        n = len(self.layers)
        for i in range(n-2):
            if self.activation_func == "sigmoid":
                self.activations[i+1] = self.sigmoid(np.matmul(self.weights[i].T,self.activations[i])+self.biases[i])
            elif self.activation_func == "tanh":
                self.activations[i+1] = self.tanh(np.matmul(self.weights[i].T,self.activations[i])+self.biases[i])
            elif self.activation_func == "relu":
                self.activations[i+1] = self.relu(np.matmul(self.weights[i].T,self.activations[i])+self.biases[i])

        self.activations[n-1] = self.softmax(np.matmul(self.weights[n-2].T,self.activations[n-2])+self.biases[n-2])

        return np.argmax(self.activations[-1])

In [ ]:
nn = Neural_network(x_train,y_train,784,32,2,10,learning_rate=0.0001,batch_size = 256,epochs=3,
                    activation_func="tanh",optimizer="adam",weight_init="random",decay_rate=0.8,loss="cross_entropy")

Epoch--- 1   loss =  4.410875810387893   accuracy =  0.285    validation loss=  3.267206881955507   validation accuaracy=  0.289
Epoch--- 2   loss =  2.6746176304116274   accuracy =  0.449    validation loss=  2.3424720007002713   validation accuaracy=  0.443
Epoch--- 3   loss =  2.0974903848117274   accuracy =  0.539    validation loss=  1.9622489860578056   validation accuaracy=  0.532


In [ ]:
nn = Neural_network(x_train,y_train,784,32,2,10,learning_rate=0.0001,batch_size = 256,epochs=3,
                    activation_func="tanh",optimizer="adam",weight_init="random",decay_rate=0.8,loss="squared_loss")

Epoch--- 1   loss =  1.0322509063777892   accuracy =  0.331    validation loss=  0.8902181314833233   validation accuaracy=  0.333
Epoch--- 2   loss =  0.8218366202268449   accuracy =  0.422    validation loss=  0.7631890619922613   validation accuaracy=  0.414
Epoch--- 3   loss =  0.7080123265647982   accuracy =  0.491    validation loss=  0.6725003234893882   validation accuaracy=  0.482


**Here in the above two cells, in the first cell we computed with loss as cross entropy for 3 epochs then we get validation accuracy of 53.2% and for the same configuration if we use loss as squared loss then we get accuracy of 48.2%**

In [ ]:
nn = Neural_network(x_train,y_train,784,16,3,10,learning_rate=0.0001,batch_size = 64,epochs=3,
                    activation_func="tanh",optimizer="nadam",weight_init="xavier",decay_rate=0.8,loss="cross_entropy")

Epoch--- 1   loss =  2.840050576380326   accuracy =  0.315    validation loss=  2.497627981540466   validation accuaracy=  0.311
Epoch--- 2   loss =  2.323451287621075   accuracy =  0.535    validation loss=  2.1520652586860063   validation accuaracy=  0.529
Epoch--- 3   loss =  1.9680381998060257   accuracy =  0.549    validation loss=  1.8275632717325387   validation accuaracy=  0.546


In [ ]:
nn = Neural_network(x_train,y_train,784,16,3,10,learning_rate=0.0001,batch_size = 64,epochs=3,
                    activation_func="tanh",optimizer="nadam",weight_init="xavier",decay_rate=0.8,loss="squared_loss")

Epoch--- 1   loss =  0.862686059173855   accuracy =  0.289    validation loss=  0.8153129325006884   validation accuaracy=  0.287
Epoch--- 2   loss =  0.778524200603818   accuracy =  0.438    validation loss=  0.7393067923006332   validation accuaracy=  0.428
Epoch--- 3   loss =  0.6907123551506595   accuracy =  0.529    validation loss=  0.646048442587927   validation accuaracy=  0.529


**Here in the above two cells, in the first cell we computed with loss as cross entropy for 3 epochs then we get validation accuracy of 54.6% and for the same configuration if we use loss as squared loss then we get accuracy of 52.9%**

In [30]:
nn = Neural_network(x_train,y_train,784,64,2,10,learning_rate=3e-3,batch_size = 64,epochs=3,
                    activation_func="relu",optimizer="adam",weight_init="xavier",decay_rate=0.9,loss="cross_entropy")

Epoch--- 1   loss =  0.872479811689399   accuracy =  0.826    validation loss=  0.6899052297527019   validation accuaracy=  0.816
Epoch--- 2   loss =  0.5919632520375588   accuracy =  0.857    validation loss=  0.5877115601479391   validation accuaracy=  0.847
Epoch--- 3   loss =  0.5355915003189532   accuracy =  0.868    validation loss=  0.546800380734861   validation accuaracy=  0.862


In [29]:
nn = Neural_network(x_train,y_train,784,64,2,10,learning_rate=3e-3,batch_size = 64,epochs=3,
                    activation_func="relu",optimizer="adam",weight_init="xavier",decay_rate=0.9,loss="squared_loss")

Epoch--- 1   loss =  0.3264697301892217   accuracy =  0.839    validation loss=  0.24180666089717034   validation accuaracy=  0.829
Epoch--- 2   loss =  0.21795579690597414   accuracy =  0.85    validation loss=  0.22580669811185336   validation accuaracy=  0.841
Epoch--- 3   loss =  0.19817767856436266   accuracy =  0.839    validation loss=  0.24834383598369933   validation accuaracy=  0.825


**bold Here in the above two cells, in the first cell we computed with loss as cross entropy for 3 epochs then we get validation accuracy of 86.2% and for the same configuration if we use loss as squared loss then we get accuracy of 82.5%text**

In [32]:
nn = Neural_network(x_train,y_train,784,64,2,10,learning_rate=2e-3,batch_size= 64,epochs=6,
                    activation_func="tanh",optimizer="nadam",weight_init="xavier",decay_rate=0.1,loss="cross_entropy")

Epoch--- 1   loss =  0.8026560179978357   accuracy =  0.856    validation loss=  0.5974650405768595   validation accuaracy=  0.842
Epoch--- 2   loss =  0.5472850285954772   accuracy =  0.869    validation loss=  0.5443255065183352   validation accuaracy=  0.855
Epoch--- 3   loss =  0.4970527330993899   accuracy =  0.875    validation loss=  0.5257676483428697   validation accuaracy=  0.86
Epoch--- 4   loss =  0.465177442865648   accuracy =  0.883    validation loss=  0.5032145444455606   validation accuaracy=  0.867
Epoch--- 5   loss =  0.44067019984154865   accuracy =  0.887    validation loss=  0.48671335359921153   validation accuaracy=  0.875
Epoch--- 6   loss =  0.4213724798032104   accuracy =  0.891    validation loss=  0.4873435136591763   validation accuaracy=  0.877


In [33]:
nn = Neural_network(x_train,y_train,784,64,2,10,learning_rate=2e-3,batch_size= 64,epochs=6,
                    activation_func="tanh",optimizer="nadam",weight_init="xavier",decay_rate=0.1,loss="squared_loss")

Epoch--- 1   loss =  0.29698845337618457   accuracy =  0.851    validation loss=  0.22552089670677541   validation accuaracy=  0.838
Epoch--- 2   loss =  0.204455888607373   accuracy =  0.857    validation loss=  0.217656674040274   validation accuaracy=  0.844
Epoch--- 3   loss =  0.1868245100858147   accuracy =  0.863    validation loss=  0.2147361265814986   validation accuaracy=  0.847
Epoch--- 4   loss =  0.17644376709351361   accuracy =  0.868    validation loss=  0.20919306435838086   validation accuaracy=  0.855
Epoch--- 5   loss =  0.16864282594379246   accuracy =  0.873    validation loss=  0.20128879951126222   validation accuaracy=  0.859
Epoch--- 6   loss =  0.16265461932845374   accuracy =  0.874    validation loss=  0.20360691831269295   validation accuaracy=  0.856


**Here in the above two cells, in the first cell we computed with loss as cross entropy for 6 epochs then we get validation accuracy of 87.7% and for the same configuration if we use loss as squared loss then we get accuracy of 85.6%**